In [405]:
from config import api_key_qwen
import mysql.connector
from mysql.connector import Error
import json
from utils import logger, clean_create_statement
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import Chroma

# Database connection configuration
DB_CONFIG = {
    "host": "gz-cdb-5scrcjb5.sql.tencentcdb.com",
    "user": "db",
    "password": "dbdb905905",
    "database": "sele",
    "port": 63432
}

def connect_db():
    """Connect to the MySQL database."""
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        if connection.is_connected():
            return connection
    except Error as e:
        logger.error(f"Error while connecting to MySQL: {e}")
        return None

def sql_query(query: str):
    """Execute an SQL query and return the results."""
    connection = connect_db()
    if not connection:
        return "Database connection failed."
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        formatted_result = ", ".join([str(row) for row in result])
    finally:
        cursor.close()
        connection.close()
    return formatted_result

def handle_tool_calls(raw_response):
    """Handle tool calls from the AI response."""
    tool_response = None
    if 'tool_calls' in raw_response.output.choices[0].message:
        for tool_call in raw_response.output.choices[0].message.tool_calls:
            if tool_call['function']['name'] == 'sql_query':
                query_args = json.loads(tool_call['function']['arguments'])
                function_result = sql_query(query_args['query'])
                tool_response = {
                    "role": "tool",
                    "content": function_result,
                    "tool_call_id": tool_call.get('id', 'N/A')
                }
                break
    return tool_response

def table_schema():
    """Generate the schema of a specific table (hardcoded for 'tender_key_detail_copy')."""
    connection = connect_db()
    if not connection:
        return "Connection error."
    cursor = connection.cursor()
    cursor.execute("SHOW CREATE TABLE tender_key_detail_copy")
    create_info = cursor.fetchone()
    return clean_create_statement(create_info[1])


def column_comments():
    """Return column comments as a JSON string."""
    columns = {
        'tender_id': '招标项目ID',
        'bid_price': '招标价格（元）',
        'construction_duration': '工期（天）',
        'construction_area': '建筑面积（平方米）',
        'construction_cost': '建安费（元）',
        'qualification_type': '监理企业资质类型,必须是来自于以下：房屋建筑工程、冶炼工程、矿山工程、化工石油工程、水利水电工程、电力工程、农林工程、铁路工程、公路工程、港口与航道工程、航天航空工程、通信工程、市政公用工程、机电安装工程，综合资质',
        'qualification_level': '监理企业资质等级',
        'qualification_profession': '总监注册资格证书专业',
        'title_level': '总监职称等级',
        'education': '总监学历',
        'performance_requirements': '总监相关业绩要求,例如，“至少担任过2项类似工程的监理负责人”',
        'simultaneous_projects_limit': '总监兼任项目限制,例如，“在任职期间能参与的其他在施项目不得超过2个”',
        'qualification_profession_addition': '附加信息',
        'title': '项目名称',
        'publish_date': '招标公告发布日期', 
        'province': '所在省份，例如“广东省”，不包含省级之外的地级市信息',
        'detail_link': '详情链接'
    }    
    return json.dumps(columns, indent=2, ensure_ascii=False)

def sample_entries(question):
    """Retrieve sample entries based on a question."""
    embeddings = DashScopeEmbeddings(model="text-embedding-v1", dashscope_api_key=api_key_qwen)
    persist_dir = "updated_tender_vector_store"
    vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    results = vectorstore.similarity_search(question, k=3)
    
    full_output = ""
    connection = connect_db()
    if connection:
        cursor = connection.cursor()
        try:
            # 获取并输出列名
            cursor.execute("SELECT * FROM tender_key_detail_copy WHERE FALSE")
            columns = [desc[0] for desc in cursor.description]
            header = ', '.join(columns)
            full_output += f"{header}\n"

            for result in results:
                # 确保之前的结果已经被完全处理
                while cursor.nextset():
                    pass
                
                query = f"SELECT * FROM tender_key_detail_copy WHERE tender_id = '{result.metadata['tender_id']}'"
                cursor.execute(query)
                sample_contents = cursor.fetchall()
                
                for row in sample_contents:
                    formatted_row = ', '.join(map(str, row))
                    full_output += f"{formatted_row}\n"

        finally:
            cursor.close()
    connection.close()
    return full_output

def sqlAgent(user_input):
    """Main function to interact with the SQL database based on user input."""
    tools = [
        {
        "type": "function",
        "function": {
            "name": "sql_query",
            "description": "此函数负责执行传入的MySQL查询语句，并返回查询结果。它专门用于检索数据，不支持修改、删除或更新数据库的操作。这包括但不限于仅执行SELECT查询语句。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "description": "一个有效的MySQL SELECT查询语句，用于从数据库中检索数据。",
                        "type": "string"
                    }
                },
                "required": ["query"]
            },
        },
        }
    ]
    
    system_prompt = """
    基于用户的问题及元数据表的结构信息，使用sql_query工具去查询可回答问题的数据。
    注意：不用直接输出，而是调用“sql_query”函数，传入查询语句。
    逐步去思考，拆解问题关键词，查看相关注释，再构建查询语句。
    """
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"用户的问题是: {user_input}"},
        {"role": "user", "content": f"元数据表的结构： {table_schema()}"},
        {"role": "user", "content": f"元数据注释： {column_comments()}"},
        {"role": "user", "content": f"元数据表数据示例： {sample_entries(user_input)}"}
    ]
    
    import dashscope
    dashscope.api_key = api_key_qwen
    model_name = "qwen-plus"
    temperature = 0.2
    
    logger.info('AI model initialized: %s', model_name)
    
    raw_response = dashscope.Generation.call(
        messages=messages,
        model=model_name,
        tools=tools,
        temperature=temperature,
        result_format='message'
    )
    # 如果没有call 函数怎么处理？
    query_call = raw_response.output.choices[0].message.tool_calls[0]
    query = query_call['function']['arguments']
    query_result = handle_tool_calls(raw_response)['content'] if handle_tool_calls(raw_response) else "No result."

    return query, query_result


In [608]:
user_input = "4月有哪些市政甲级资质可以投的标？"

# sample_entries(user_input)
aa,bb = sqlAgent(user_input)

In [609]:
aa

'{"query": "SELECT * FROM tender_key_detail_copy WHERE publish_date BETWEEN \'2024-04-01\' AND \'2024-04-30\' AND qualification_type = \'市政公用工程\' AND qualification_level = \'甲级\'"}'

In [610]:
bb

"(525, 1179860.0, 1095, 10000.0, 69335000.0, '市政公用工程', '甲级', '市政公用工程', None, None, None, None, None, '杏坛镇齐赞路改道工程监理', datetime.date(2024, 4, 2), '广东', 'http://www.ggzy.gov.cn/information/html/a/440000/0101/202404/02/0044d38d6e7b00dd436eacd1c7d5469b50ba.shtml'), (541, 1722130.0, 720, None, 5000000.0, '市政公用工程', '甲级', '市政公用工程', None, None, None, '3', None, '东江湾产业园荔城片区市政道路升级改造工程项目（监理）【监理】招标公告', datetime.date(2024, 4, 3), '广东', 'http://www.ggzy.gov.cn/information/html/a/440000/0101/202404/03/00441a3259c145cc459fbf788ecacc78a048.shtml'), (545, 1361520.0, 450, None, 74368900.0, '市政公用工程', '甲级', '房屋建筑工程', '高级工程师', '硕士', None, None, None, '南头镇东福北路道路改造工程监理', datetime.date(2024, 4, 3), '广东', 'http://www.ggzy.gov.cn/information/html/a/440000/0101/202404/03/004454ffae6fa4a34db3874152f4a3ab4917.shtml'), (558, 21037900.0, 1460, None, None, '市政公用工程', '甲级', '市政公用工程', None, None, None, None, None, '顺德粮食产业园周边市政道路工程（二期）监理（二次）', datetime.date(2024, 4, 3), '广东', 'http://www.ggzy.gov.cn/information/html/a/4400

In [603]:
# 现在要写一个AI agent， sql_retrieval_grader 判断上述query 和 query result 是否和问题相关

def sql_retrieval_grader(user_input, query, query_result):
    """
    使用DashScope的大模型评估SQL查询及其结果与用户问题的相关性。
    """
    import dashscope

    # 设置DashScope API Key
    dashscope.api_key = api_key_qwen

    # 构建用于评估的prompt字符串，确保清晰、完整地传达了查询、结果与原始问题
    evaluation_prompt = (
    f"用户询问：'{user_input}'\n"
    f"为此问题设计的SQL查询是：\n{query}\n"
    f"执行的SQL查询产生了以下结果概要：\n"
    f"{('查询结果为空。' if not query_result else query_result)}\n"
    "\n请严格判断，此查询结果是否完美对应并能精确解答用户的问题。"
    "若结果完全符合，回答'是'；若有任何偏差、遗漏或信息不足之处，包括结果为空，请回答'否'。"
    "\n请仅回答'是'或'否'，无需额外解释。"
    )


    # 调用DashScope模型进行评估
    model_name = "qwen-plus"
    temperature = 0.2
    raw_response = dashscope.Generation.call(
        messages=[{"role": "user", "content": evaluation_prompt}],
        model=model_name,
        temperature=temperature,
        result_format='message'
    )

    # 解析模型的回复
    ai_response = raw_response.output.choices[0].message.content.strip().lower()

    # 判断并返回结果
    if ai_response in ['是', 'yes']:
        return True, ai_response
    elif ai_response in ['否', 'no']:
        return False, ai_response
    else:
        return None, ai_response


In [611]:
print(sql_retrieval_grader(user_input, aa, bb))

(True, '是')


In [612]:
def sql_retrieval_refine(user_input, query, query_result):
    """
    该函数旨在利用DashScope的大规模语言模型优化检索结果，
    根据用户的具体问题，从SQL查询结果中提炼并重组相关信息，
    确保返回的内容紧密相关且信息无遗漏。
    """
    
    import dashscope

    # 设置DashScope服务的API密钥
    dashscope.api_key = api_key_qwen

    # 构建指导模型评估的指令，明确了用户的问题、设计的SQL查询及查询结果，
    # 并指示模型需精确提炼与用户问题直接相关的数据细节。
    evaluation_prompt = (
        f"用户提问核心为：'{user_input}'\n"
        f"为此定制的SQL查询语句为：\n{query}\n"
        f"该查询返回的数据概览如下：\n{query_result}\n"
        "\n任务要求：从上述数据中，依据用户提问的实质，精心筛选并重新组织信息，"
        "务必确保所有紧要数据均被充分阐释，无遗漏。"
    )

    # 调用DashScope的高级模型进行信息提炼和重组，
    # 选用模型$qwen-plus$以期获得高质量的响应，同时设置生成温度以控制创意程度。
    model_name = "qwen-plus"
    temperature = 0.2
    raw_response = dashscope.Generation.call(
        messages=[{"role": "user", "content": evaluation_prompt}],
        model=model_name,
        temperature=temperature,
        result_format='message'
    )

    # 从模型响应中提取并返回最终的精炼信息，
    # 此内容应全面覆盖用户查询需求中的所有关键点。
    ai_response = raw_response.output.choices[0].message.content
    
    return ai_response

In [613]:
sql_retrieval_refine(user_input, aa, bb)

'在2024年4月，广东省有多个市政甲级资质的工程项目正在招标。以下是一些具体的项目信息：\n\n1. 杏坛镇齐赞路改道工程监理：该项目于4月2日发布，需要市政公用工程甲级资质。\n\n2. 东江湾产业园荔城片区市政道路升级改造工程项目（监理）：4月3日发布，同样需要市政公用工程甲级资质。\n\n3. 南头镇东福北路道路改造工程监理：4月3日发布，除了市政公用工程甲级资质，还要求高级工程师资格。\n\n4. 顺德粮食产业园周边市政道路工程（二期）监理（二次）：4月3日发布，需市政公用工程甲级资质。\n\n5. 深圳市城市轨道交通27号线一期工程监理27501标和27505标：分别于4月4日发布，均需市政公用工程甲级资质，且涉及铁路工程。\n\n6. 茂名市城区排涝通道改造工程（一期）监理：4月9日发布，要求市政公用工程甲级资质。\n\n7. 鹅埠片区市政路网建设工程（龙山大道、龙颐路）监理服务：4月9日发布，需市政公用工程甲级资质。\n\n8. 大亚湾石化扩展区市政配套设施建设项目（二期）一区工程监理：4月10日发布，要求市政公用工程甲级资质。\n\n9. 虎地排一路、虎地排二路（监理批量招标）：4月12日发布，需市政公用工程甲级资质。\n\n10. 天河体育中心及华南农业大学体育馆周边区域市政设施功能完善和品质提升工程监理：4月13日发布，要求市政公用工程甲级资质。\n\n11. 黄埔大道东（科韵路-汇彩路）周边区域市政设施功能完善和品质提升工程监理：4月13日发布，需市政公用工程甲级资质。\n\n12. 水贝时尚设计总部经济集聚区街区品质提升项目（一期）监理：4月15日和16日两次发布，均需市政公用工程甲级资质。\n\n13. 龙山四路（塘横大道-新荷大道段）市政工程监理：4月18日发布，需市政公用工程甲级资质。\n\n14. 茂名市中心城区（北组团）城市内涝治理项目（一期）监理：4月20日发布，要求市政公用工程甲级资质。\n\n15. 揭东区中国玉都基础设施提升建设项目工程监理：4月22日发布，需市政公用工程甲级资质。\n\n16. 佛山市顺德区伦教三洲东部产业城提升改造工程监理：4月22日发布，要求市政公用工程甲级资质。\n\n17. 深圳市城市轨道交通19号线一期工程监理19504标和19501标：4月23日发布，需市政公用工程甲级资质，且涉及铁路工程。\

In [ ]:
# 回答Agent，很简单
# 输入 ‘用户问题’， ‘SQL查询’， ‘SQL查询结果’
# 系统额外要求：规定输出的形式
# 输出 ‘回答’

In [332]:
# 现将元数据表（tender_key_detail_copy）内容embedding
import shutil
from datetime import date

import os
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import Chroma

def embed_and_persist_data(api_key='sk-9811d3c24a0f4ff99c63733a080f3aef', persist_dir="metadata_copy_vector_store"):
    """
    从数据库提取数据，构造中文键值对的公告数据，进行文本嵌入，并持久化到Chroma向量数据库中。
    """
    def sql_query(query):
        try:
            connection = connect_db()
            cursor = connection.cursor()
            cursor.execute(query)
            results = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]  # 获取列名
            cursor.close()
            connection.close()
            return columns, results  # 确保这里返回列名和结果
        except mysql.connector.Error as err:
            print(f"Database Error occurred: {err}")
            return None, None  


    query = "SELECT * FROM tender_key_detail_copy;"
    columns, raw_results = sql_query(query)
    if columns is None or raw_results is None:
        print("数据库查询失败，无法继续执行。")
        return


    chinese_columns = {
        'tender_id': '招标项目ID',
        'bid_price': '招标价格（元）',
        'construction_duration': '工期（天）',
        'construction_area': '建筑面积（平方米）',
        'construction_cost': '建安费（元）',
        'qualification_type': '监理企业资质类型',
        'qualification_level': '监理企业资质等级',
        'qualification_profession': '总监注册资格证书专业',
        'title_level': '总监职称等级',
        'education': '总监学历',
        'performance_requirements': '总监相关业绩要求',
        'simultaneous_projects_limit': '总监兼任项目限制',
        'qualification_profession_addition': '附加信息',
        'title': '项目名称',
        'publish_date': '公告发布日期', 
        'province': '所在省份',
        'detail_link': '详情链接'
    }    
    results = [list(row) for row in raw_results]
    chinese_column_names = [chinese_columns[col] if col in chinese_columns else col for col in columns]

    tender_announcements = []
    for row in results:
        announcement = {}
        for i, value in enumerate(row):
            if isinstance(value, date):  # 检查是否为日期类型
                value = value.strftime('%Y-%m-%d')  # 转换为字符串格式
            announcement[chinese_column_names[i]] = value
        tender_announcements.append(announcement)

    # 准备嵌入数据和元数据
    texts = [json.dumps(announcement,ensure_ascii=False) for announcement in tender_announcements]
    metadata_list = [{"tender_id": announcement["招标项目ID"]} for announcement in tender_announcements]


    # 初始化 DashScope 嵌入模型
    embeddings = DashScopeEmbeddings(
        model="text-embedding-v1", dashscope_api_key=api_key
    )

    # 安全删除旧的持久化目录（如果存在），避免权限问题
    if os.path.exists(persist_dir):
        shutil.rmtree(persist_dir)
    
    # 确保持久化目录存在
    os.makedirs(persist_dir, exist_ok=True)



    # 初始化向量存储，并添加嵌入向量和元数据
    vectorstore = Chroma.from_texts(
        texts=texts,  
        embedding=embeddings, 
        metadatas=metadata_list,  
        persist_directory=persist_dir
    )

    print("数据嵌入及持久化完成。")
    return vectorstore


In [333]:
# 现在每次不得不用新的路径，要解决一下这个权限问题

persist_dir="metadata_copy_vector_store"

embed_and_persist_data(api_key=api_key_qwen, persist_dir=persist_dir)

数据嵌入及持久化完成。


In [414]:
def retrieval_planner(user_input):
    """Main function to interact with the SQL database based on user input."""
    tools = [
        {
        "type": "function",
        "function": {
            "name": "sql_query",
            "description": "此函数负责执行传入的MySQL查询语句，并返回查询结果。它专门用于检索数据，不支持修改、删除或更新数据库的操作。这包括但不限于仅执行SELECT查询语句。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "description": "一个有效的MySQL SELECT查询语句，用于从数据库中检索数据。",
                        "type": "string"
                    }
                },
                "required": ["query"]
            },
        },
        },
        {
        "type": "function",
        "function": {
            "name": "sematic_search",
            "description": "此函数基于语义搜索技术，从招标文件中检索与用户问题相关的信息。同时执行元数据表筛选，以及相关内容的语义搜索。",
            "parameters": {
                "type": "object",
                "properties": {
                    "filter": {
                        "description": "要基于元数据筛选的内容，可以直接自然语言描述。",
                        "type": "string"
                    },
                    "keywords": {
                        "description": "要基于语义搜索的内容，可以直接自然语言描述。",
                        "type": "string"
                    }
                },
                "required": ["keywords"]
            },
        },
        }

    ]
    
    system_prompt = """
    基于元数据的表结构信息，判断用户问题是否可以通过元数据表中的数据回答。
    如果可以，使用sql_query工具去查询可回答问题的数据。
    如果不行，则要通过语义查询(sematic search)招标文件来回答，基于问题及元数据结构，将要用元数据筛选的条件信息和要用语义搜索的关键词信息传入sematic_search函数。
    """
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"用户的问题是: {user_input}"},
        {"role": "user", "content": f"元数据表的结构： {table_schema()}"},
        {"role": "user", "content": f"元数据注释： {column_comments()}"},
        {"role": "user", "content": f"元数据表数据示例： {sample_entries(user_input)}"}
    ]
    
    import dashscope
    dashscope.api_key = api_key_qwen
    model_name = "qwen-plus"
    temperature = 0.2
    
    logger.info('AI model initialized: %s', model_name)
    
    raw_response = dashscope.Generation.call(
        messages=messages,
        model=model_name,
        tools=tools,
        temperature=temperature,
        result_format='message'
    )
    
    ai_response = raw_response.output.choices[0].message.content
    
    query_call = raw_response.output.choices[0].message.tool_calls[0]
    query = query_call['function']['arguments']
    query_result = handle_tool_calls(raw_response)['content'] if handle_tool_calls(raw_response) else "No result."

    return query, query_result,ai_response


In [415]:
question = "4月有哪些市政甲级资质可以投的标？并且告诉我这些标的开标日期以及开标地点"

aa,bb,cc = retrieval_planner(question)

In [416]:
print(aa)
print(bb)

{"query": "SELECT * FROM tender_key_detail_copy WHERE qualification_type = '市政公用工程' AND qualification_level = '甲级' AND YEAR(publish_date) = 2024 AND MONTH(publish_date) = 4"}
(525, 1179860.0, 1095, 10000.0, 69335000.0, '市政公用工程', '甲级', '市政公用工程', None, None, None, None, None, '杏坛镇齐赞路改道工程监理', datetime.date(2024, 4, 2), '广东', 'http://www.ggzy.gov.cn/information/html/a/440000/0101/202404/02/0044d38d6e7b00dd436eacd1c7d5469b50ba.shtml'), (541, 1722130.0, 720, None, 5000000.0, '市政公用工程', '甲级', '市政公用工程', None, None, None, '3', None, '东江湾产业园荔城片区市政道路升级改造工程项目（监理）【监理】招标公告', datetime.date(2024, 4, 3), '广东', 'http://www.ggzy.gov.cn/information/html/a/440000/0101/202404/03/00441a3259c145cc459fbf788ecacc78a048.shtml'), (545, 1361520.0, 450, None, 74368900.0, '市政公用工程', '甲级', '房屋建筑工程', '高级工程师', '硕士', None, None, None, '南头镇东福北路道路改造工程监理', datetime.date(2024, 4, 3), '广东', 'http://www.ggzy.gov.cn/information/html/a/440000/0101/202404/03/004454ffae6fa4a34db3874152f4a3ab4917.shtml'), (558, 21037900.0, 1460, Non

In [417]:
print(cc)

根据提供的元数据表结构，我们可以看到`qualification_type`字段表示监理企业资质类型，`qualification_level`字段表示监理企业资质等级。用户询问的是4月有市政甲级资质可以投标的项目，因此我们需要筛选出`qualification_type`为'市政公用工程'且`qualification_level`为'甲级'的记录，并且`publish_date`在4月份的项目。

让我们先执行SQL查询来获取这些信息。


In [614]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

def fetch_html_data():
    connection = connect_db()
    if connection is not None:
        try:
            # 更新查询语句以获取tender_key_detail_copy表的所有列以及td.cleaned_detail_html
            # 注意：这里的tkd.*代表tender_key_detail_copy表的所有列，具体列名请根据实际情况调整
            query = """
            SELECT td.id, td.tender_id, td.cleaned_detail_html, tkd.*
            FROM tender_detail_html td
            JOIN tender_key_detail_copy tkd ON td.tender_id = tkd.tender_id
            """
            # 根据需要可以加入LIMIT条件来限制查询结果数量，或完全移除以获取全部数据
            df = pd.read_sql(query, connection)
            connection.close()
            return df
        except Error as e:
            logger.error(f"Error fetching data from MySQL: {e}")
            connection.close()
            return None
    else:
        return None
# 调用函数获取数据
data_df = fetch_html_data()
if data_df is not None:
    print("Data fetched successfully.")
else:
    print("Failed to fetch data.")

/var/folders/1x/ztlmp2z97k52zb7w09c9nq4c0000gn/T/ipykernel_95513/3562367000.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


Data fetched successfully.


In [615]:
data_df.shape

(658, 20)

In [620]:
sample_html = data_df.loc[111, 'cleaned_detail_html']

In [621]:
print(len(sample_html))
print(sample_html)


3370
  <div><div><div><table><tbody><tr><td><p>投资项目代码</p></td><td><p>2210-440784-04-01-974729</p></td></tr><tr><td><p>投资项目名称</p></td><td><p>鹤山市沙坪街道水利基础设施建设项目</p></td></tr><tr><td><p>招标项目名称</p></td><td><p>鹤山市沙坪街道水利基础设施建设项目—鹤山市沙坪街道截污工程监理</p></td></tr><tr><td><p>标段(包)名称</p></td><td><p>/</p></td><td><p>公告性质</p></td><td><p>正常</p></td></tr><tr><td><p>资格审查方式</p></td><td><p>资格后审</p></td></tr><tr><td><p>招标项目实施</p><p>(交货)地点</p></td><td><p>鹤山市沙坪街道</p></td></tr><tr><td><p>资金来源</p></td><td><p>其它</p></td><td><p>资金来源构成</p></td><td><p>除申请专项债券资金外，其余由鹤山市沙坪街道办事处筹集解决</p></td></tr><tr><td><p>招标范围及规模</p></td><td><p>本工程项目准备阶段、施工阶段、竣工结算阶段及工程质量保修阶段的质量控制、安全生产监督管理、投资控制、进度控制、合同管理、信息管理、组织协调等监理工作以及缺陷责任期内的监理服务和相关专业技术咨询服务。</p></td></tr><tr><td><p>招标内容</p></td><td><p>沙坪街道汇源谷埠市场片区（含谷埠市场部分和鹤山大道汇源至赤坎汇源渠段部分）、楼冲上中社开发区、镇南工业区、荀山工业区、莺朗工业区、玉桥朗围产业聚集区、桥氹工业聚集区以及黄宝坑工业区8个项目区产生的生活污水，主要建设任务为对以上8个项目区产生的生活污水纳管就近接入现状已建市政污水管网，最终进入污水处理厂达标处理排放。排水体制为雨污分流。</p></td></tr><tr><td><p>工期(交货期)</p></td><td><p>施工准备阶段、施工阶段（暂定547日历天）、竣工结算阶段及工程质量保修阶段全过

In [622]:
from bs4 import BeautifulSoup

# 假设html_sample是您提供的HTML样本
soup = BeautifulSoup(sample_html, 'html.parser')

# 定义一个函数来处理不同类型的标签
# def handle_tag(tag):
#     if tag.name == 'p':
#         # 处理段落，这里可以是分割、清理或直接收集文本
#         yield "" + tag.text.strip() + ""
#     elif tag.name.startswith('h'):
#         # 处理标题，可能意味着一个新的内容段落开始
#         yield "标题: " + tag.text.strip() + "\n"
#     # 可以继续添加对其他标签的处理逻辑

def handle_tag(tag):
    if tag.name == 'p':
        # 处理段落
        yield "" + tag.text.strip() + ""
    elif tag.name.startswith('h'):
        # 处理标题
        yield "标题: " + tag.text.strip() + "\n"
    elif tag.name == 'table':
        # 新增：处理表格
        for row in tag.find_all('tr'):  # 遍历表格的每一行
            row_data = [cell.text.strip() for cell in row.find_all(['th', 'td'])]  # 获取每个单元格的内容
            yield ", ".join(row_data) + "\n"  # 输出处理过的行数据，这里以逗号分隔作为示例
    else:
        # 处理其他未特定说明的标签，根据需要可以是简单的文本内容收集或其他逻辑
        if tag.string:  # 检查标签是否有直接的文本内容
            yield tag.text.strip() + "\n"




In [623]:
##  手动测试
# 遍历文档，应用处理函数
chunks = []
for tag in soup.descendants:
    if isinstance(tag, str):
        continue  # 跳过文本节点，因为我们已经在处理tag时包含了它们
    chunk = handle_tag(tag)
    if chunk:
        chunks.extend(chunk)

# 现在，chunks包含根据HTML结构分隔的内容块

print(len("\n".join(chunks)))

print("\n".join(chunks))



4402
投资项目代码, 2210-440784-04-01-974729

投资项目名称, 鹤山市沙坪街道水利基础设施建设项目

招标项目名称, 鹤山市沙坪街道水利基础设施建设项目—鹤山市沙坪街道截污工程监理

标段(包)名称, /, 公告性质, 正常

资格审查方式, 资格后审

招标项目实施(交货)地点, 鹤山市沙坪街道

资金来源, 其它, 资金来源构成, 除申请专项债券资金外，其余由鹤山市沙坪街道办事处筹集解决

招标范围及规模, 本工程项目准备阶段、施工阶段、竣工结算阶段及工程质量保修阶段的质量控制、安全生产监督管理、投资控制、进度控制、合同管理、信息管理、组织协调等监理工作以及缺陷责任期内的监理服务和相关专业技术咨询服务。

招标内容, 沙坪街道汇源谷埠市场片区（含谷埠市场部分和鹤山大道汇源至赤坎汇源渠段部分）、楼冲上中社开发区、镇南工业区、荀山工业区、莺朗工业区、玉桥朗围产业聚集区、桥氹工业聚集区以及黄宝坑工业区8个项目区产生的生活污水，主要建设任务为对以上8个项目区产生的生活污水纳管就近接入现状已建市政污水管网，最终进入污水处理厂达标处理排放。排水体制为雨污分流。

工期(交货期), 施工准备阶段、施工阶段（暂定547日历天）、竣工结算阶段及工程质量保修阶段全过程监理服务（含缺陷责任期）。

最高投标限价, 189.91万元

是否接受联合体投标, 否

投标资格能力要求(包括但不限于资质人员、业绩等要求), 投标人资格要求, 须具备工程监理综合类资质或市政公用工程监理甲级资质，/业绩，并在人员、试验检测仪器设备方面具有相应的监理能力。

投标人业绩要求, /

是否采用电子招标投标方式, 否, 获取资格预审/招标文件的方式, 下载资格预审/招标文 件的网络地址, 广东省公共资源交易平台(https://ygp.gdzwfw.gov.cn/ggzy-portal/index.html#/440700/index)

获取纸质资格预审/招 标文件的方式, /

获取资格预审/招标文 件开始时间, 2024年2月6日, 获取资格预审/招标文件截止时间, 2024年2月19日17时30分

投资项目代码

投资项目代码
2210-440784-04-01-974729

2210-440784-04-01-974729
投资项目名称

投资项目名称
鹤山

In [624]:
import re

def split_into_chunks_by_sentence_and_threshold(text, max_threshold):
    """
    使用正则表达式进行简单的中文句子分割，并确保每个分割后的句子组成的块不超过max_threshold
    """
    # 基于常见的中文句末标点进行分割
    sentences = re.split(r'(?<=[。！？\?])\s*', text)
    # 过滤空字符串
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    chunks = []  # 用于存储符合长度要求的句子块
    current_chunk = ""
    current_length = 0
    
    for sentence in sentences:
        sentence_length = len(sentence)
        # 如果加上当前句子会超过max_threshold，则处理当前chunk并开始新的chunk
        if current_length + sentence_length > max_threshold and current_length > 0:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_length = sentence_length
        else:
            if current_chunk:
                current_chunk += " " + sentence
            else:
                current_chunk = sentence
            current_length += sentence_length + 1  # 加1是因为句子间的空格
        
        # 单独的句子超过max_threshold，直接添加
        if sentence_length > max_threshold:
            chunks.append(sentence)
            current_chunk = ""
            current_length = 0
            
    # 添加最后一个chunk
    if current_chunk:
        chunks.append(current_chunk.strip())
        
    return chunks

def combine_paragraphs_v6(paragraphs, min_threshold, max_threshold):
    """
    根据字数阈值合并段落，并确保每个chunk的字数在min_threshold和max_threshold之间。
    对于不足min_threshold的段落，尝试与后续段落合并，直到达到min_threshold。
    BTW：实际上没有，现在就是几乎到min_threshold，也就是加上下一段到min_threshold。不想再debug了，暂用
    对于超过max_threshold的段落，进一步按句子分割处理。
    """
    combined_chunks = []
    current_chunk = ""
    current_length = 0

    for para in paragraphs:
        para_text = para.strip()
        para_length = len(para_text)

        # 尝试合并短段落
        if current_length + para_length < min_threshold:
            if current_chunk:
                current_chunk += " " + para_text
            else:
                current_chunk = para_text
            current_length += para_length + 1  # 加1是因为句子间的空格
        else:
            # 处理已达到或超过min_threshold的chunk
            if current_chunk:
                combined_chunks.append(current_chunk.strip())
                current_chunk = ""
                current_length = 0
            
            # 直接添加长度符合条件的段落
            if para_length >= min_threshold and para_length <= max_threshold:
                combined_chunks.append(para_text)
            # 对超过max_threshold的段落按句子分割处理
            elif para_length > max_threshold:
                sentence_chunks = split_into_chunks_by_sentence_and_threshold(para_text, max_threshold)
                combined_chunks.extend(sentence_chunks)
            else:
                # 当前段落小于min_threshold，开始一个新的chunk
                current_chunk = para_text
                current_length = para_length
                
    # 添加最后一个未完成的chunk，如果它达到了最小长度
    if current_chunk and current_length >= min_threshold:
        combined_chunks.append(current_chunk.strip())

    return combined_chunks

# 再次使用相同的段落和阈值测试修正后的函数
combined_chunks_v6 = combine_paragraphs_v6(chunks, 300, 400)
sum_len = 0
for chunk in combined_chunks_v6:
    print(f"{'-'*20}\n{chunk}")
    print(f"chunk长度：{len(chunk)}")
    sum_len = sum_len + len(chunk)
print(f"总长度：{sum_len}")

--------------------
投资项目代码, 2210-440784-04-01-974729 投资项目名称, 鹤山市沙坪街道水利基础设施建设项目 招标项目名称, 鹤山市沙坪街道水利基础设施建设项目—鹤山市沙坪街道截污工程监理 标段(包)名称, /, 公告性质, 正常 资格审查方式, 资格后审 招标项目实施(交货)地点, 鹤山市沙坪街道 资金来源, 其它, 资金来源构成, 除申请专项债券资金外，其余由鹤山市沙坪街道办事处筹集解决
chunk长度：201
--------------------
招标范围及规模, 本工程项目准备阶段、施工阶段、竣工结算阶段及工程质量保修阶段的质量控制、安全生产监督管理、投资控制、进度控制、合同管理、信息管理、组织协调等监理工作以及缺陷责任期内的监理服务和相关专业技术咨询服务。 招标内容, 沙坪街道汇源谷埠市场片区（含谷埠市场部分和鹤山大道汇源至赤坎汇源渠段部分）、楼冲上中社开发区、镇南工业区、荀山工业区、莺朗工业区、玉桥朗围产业聚集区、桥氹工业聚集区以及黄宝坑工业区8个项目区产生的生活污水，主要建设任务为对以上8个项目区产生的生活污水纳管就近接入现状已建市政污水管网，最终进入污水处理厂达标处理排放。排水体制为雨污分流。
chunk长度：283
--------------------
工期(交货期), 施工准备阶段、施工阶段（暂定547日历天）、竣工结算阶段及工程质量保修阶段全过程监理服务（含缺陷责任期）。 最高投标限价, 189.91万元 是否接受联合体投标, 否 投标资格能力要求(包括但不限于资质人员、业绩等要求), 投标人资格要求, 须具备工程监理综合类资质或市政公用工程监理甲级资质，/业绩，并在人员、试验检测仪器设备方面具有相应的监理能力。 投标人业绩要求, /
chunk长度：196
--------------------
是否采用电子招标投标方式, 否, 获取资格预审/招标文件的方式, 下载资格预审/招标文 件的网络地址, 广东省公共资源交易平台(https://ygp.gdzwfw.gov.cn/ggzy-portal/index.html#/440700/index) 获取纸质资格预审/招 标文件的方式, / 获取资格预审/招标文 件开始时间, 2024年2月6日, 获取资格预审/招标文件截止时间, 20

In [625]:
from bs4 import BeautifulSoup

def process_html_to_chunks(df, min_threshold=300, max_threshold=400):
    """
    处理HTML内容为文本chunks，并准备元数据。
    """
    chunks_with_metadata = []
    for idx, row in df.iterrows():
        metadata = row.drop(['id', 'cleaned_detail_html']).to_dict()
        cleaned_html = row['cleaned_detail_html']
        soup = BeautifulSoup(cleaned_html, 'html.parser')
        
        all_chunks = []
        for tag in soup.descendants:
            if isinstance(tag, str):
                continue
            handled_content = handle_tag(tag)
            if handled_content:
                all_chunks.extend(handled_content)
        
        combined_paragraphs = combine_paragraphs_v6(all_chunks, min_threshold, max_threshold)
        
        for chunk in combined_paragraphs:
            chunks_with_metadata.append({"text": chunk, "metadata": metadata})
    
    return chunks_with_metadata

In [626]:
chunks_with_metadata = process_html_to_chunks(data_df, min_threshold=300, max_threshold=400)


In [653]:
len(chunks_with_metadata)
chunks_with_metadata[0]


{'text': '工程招标公告 公告时间：2024年03月01日\n至 2024年03月21日 投资项目代码：\n2202-441900-04-01-922343 工程编码（标段编码）：\nE4419000748007679001001 招标编号：\nSSBSSO12400157 投资项目名称：\n环莞快速路三期长安莲湖路至莞长路段工程 招标项目名称：\n环莞快速路三期长安莲湖路至莞长路段工程施工监理 工程（标段）名称：环莞快速路三期长安莲湖路至莞长路段工程施工监理 招标方式：公开招标 公告性质：正常公告 资格审查方式：资格后审 招标项目地点：东莞市虎门、大岭山镇 资金来源：政府投资 资金来源构成：财政投资',
 'metadata': {'tender_id': 1,
  'bid_price': 23691400.0,
  'construction_duration': 2190.0,
  'construction_area': nan,
  'construction_cost': 2192000000.0,
  'qualification_type': '公路工程',
  'qualification_level': '甲级',
  'qualification_profession': None,
  'title_level': '高级工程师',
  'education': None,
  'performance_requirements': '至少担任过2项类似工程土建工程施工监理的监理负责人',
  'simultaneous_projects_limit': None,
  'qualification_profession_addition': None,
  'title': '环莞快速路三期长安莲湖路至莞长路段工程施工监理',
  'publish_date': datetime.date(2024, 2, 29),
  'province': '广东',
  'detail_link': 'http://www.ggzy.gov.cn/information/html/a/440000/0101/202402/29/00443a5da496127044af83f79bfcc9d7285c.shtml'}}

In [579]:
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import Chroma


def embed_and_persist_tender_announcements(api_key, persist_dir, chunks_with_metadata):
    """
    基于处理好的文本chunks和元数据进行embedding并持久化。
    """
    texts = [chunk['text'] for chunk in chunks_with_metadata]
    metadata_list = [chunk['metadata'] for chunk in chunks_with_metadata]
    
    embeddings = DashScopeEmbeddings(model="text-embedding-v1", dashscope_api_key=api_key)
    vectorstore = Chroma.from_texts(
        texts=texts,  
        embedding=embeddings, 
        metadatas=metadata_list,  
        persist_directory=persist_dir
    )
    print("招标公告文本嵌入及持久化完成。")
    return vectorstore


In [658]:

from langchain.docstore.document import Document
from langchain_community.vectorstores.utils import filter_complex_metadata
from datetime import datetime,date

def process_and_filter_metadata(chunks_with_metadata):
    """
    过滤并处理元数据，先将字典转换为Document对象，使用filter_complex_metadata过滤，
    然后将结果转换回原始字典格式。
    """
    filtered_chunks = []
    for chunk in chunks_with_metadata:
        # 将字典转换为Document对象
        doc = Document(page_content=chunk["text"], metadata=chunk["metadata"])
        
        # 应用过滤器并获取过滤后的Document对象
        filtered_doc = filter_complex_metadata([doc], allowed_types=(str, int, float, bool, date, datetime))[0]
        
        # 将过滤后的Document对象转换回字典格式
        filtered_metadata = filtered_doc.metadata
        filtered_chunks.append({"text": filtered_doc.page_content, "metadata": filtered_metadata})
    
    return filtered_chunks

In [659]:

# # 调用函数获取数据
# data_df = fetch_html_data()

# # 处理HTML为文本chunks
# chunks_with_metadata = process_html_to_chunks(data_df)


# 进一步过滤和处理元数据以确保兼容性
filtered_chunks_with_metadata = process_and_filter_metadata(chunks_with_metadata)


In [701]:
filtered_chunks_with_metadata[11]
# len(filtered_chunks_with_metadata)

{'text': '项目概算总投资8854.64万元，其中建安费8058.16万元。',
 'metadata': {'tender_id': 5,
  'bid_price': 1397500.0,
  'construction_duration': 150.0,
  'construction_area': 10000.0,
  'construction_cost': 80581600.0,
  'qualification_type': '市政公用工程',
  'qualification_level': '丙级',
  'qualification_profession': '市政公用工程',
  'title': '提升新兴县“红星闪耀?富美太平”乡村振兴示范带项目（监理）招标公告',
  'publish_date': datetime.date(2024, 2, 29),
  'province': '广东',
  'detail_link': 'http://www.ggzy.gov.cn/information/html/a/440000/0101/202402/29/0044d63fedd32485408f9179bdd3d5d79865.shtml'}}

In [702]:

# 假设您已经有了API密钥和持久化目录的路径
api_key = api_key_qwen
persist_directory = "tender_chunks_vectors_v3"

# 执行embedding并持久化
vectorstore = embed_and_persist_tender_announcements(api_key, persist_directory, filtered_chunks_with_metadata)


ValueError: Expected metadata value to be a str, int, float or bool, got 2024-02-29 which is a date

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

In [684]:
vectorstore

In [699]:
# 这一步其实还有一些问题，应该有一个loading的函数，然后再 Rag。只是这里省略了。

qualification_type = "市政公用工程"

retrieval = vectorstore.similarity_search(
    query="佛山市市政工程的标有哪些？他们开标时间都是什么时候",
    k=5,
    filter = {"tender_id": 1}
    )

In [ ]:
{
  "filter": {
    "province": "广东省",
    "publish_date": {
      "$gte": "2022-04-21",
      "$lte": "2022-04-30"
    }
  }
}


In [700]:
for entry in retrieval:
    print(entry.metadata)

{'bid_price': 23691400.0, 'construction_cost': 2192000000.0, 'construction_duration': 2190.0, 'detail_link': 'http://www.ggzy.gov.cn/information/html/a/440000/0101/202402/29/00443a5da496127044af83f79bfcc9d7285c.shtml', 'performance_requirements': '至少担任过2项类似工程土建工程施工监理的监理负责人', 'province': '广东', 'qualification_level': '甲级', 'qualification_type': '公路工程', 'tender_id': 1, 'title': '环莞快速路三期长安莲湖路至莞长路段工程施工监理', 'title_level': '高级工程师'}
{'bid_price': 23691400.0, 'construction_cost': 2192000000.0, 'construction_duration': 2190.0, 'detail_link': 'http://www.ggzy.gov.cn/information/html/a/440000/0101/202402/29/00443a5da496127044af83f79bfcc9d7285c.shtml', 'performance_requirements': '至少担任过2项类似工程土建工程施工监理的监理负责人', 'province': '广东', 'qualification_level': '甲级', 'qualification_type': '公路工程', 'tender_id': 1, 'title': '环莞快速路三期长安莲湖路至莞长路段工程施工监理', 'title_level': '高级工程师'}
{'bid_price': 23691400.0, 'construction_cost': 2192000000.0, 'construction_duration': 2190.0, 'detail_link': 'http://www.ggzy.gov.cn/informati

In [677]:
metadata_schema = """ 
元数据结构:
- tender_id (Integer) - 招标项目ID
- bid_price (Float) - 招标价格（元）
- construction_duration (Integer) - 工期（天）
- construction_area (Float) - 建筑面积（平方米）
- construction_cost (Float) - 建安费（元）
- qualification_type (String) - 监理企业资质类型
- qualification_level (String) - 监理企业资质等级
- qualification_profession (String) - 总监注册资格证书专业
- title_level (String) - 总监职称等级
- education (String) - 总监学历
- performance_requirements (String) - 总监相关业绩要求
- simultaneous_projects_limit (String) - 总监兼任项目限制
- qualification_profession_addition (String) - 附加信息
- title (String) - 项目名称
- publish_date (Date) - 招标公告发布日期
- province (String) - 仅限省份名称，例如“广东省”。此字段不包括城市、区县等下级行政区划信息，确保数据筛选时聚焦于省级层面。
- detail_link (String) - 招标公告原文链接
"""


In [665]:
chroma_rule = """ 
Chroma 提供了灵活的查询和筛选功能，允许你基于文档的元数据或内容来过滤数据。例：
```python
query = "佛山市市政工程的标有哪些？"
start_date = datetime(2023, 5, 10)
end_date = datetime(2023, 6, 10)
filtered_docs = chroma_store.similarity_search(query=query,filter={"timestamp": {"$gte": start_date.isoformat(),"$lte": end_date.isoformat()}})

```
### 元数据过滤（filter）
- **目的**：仅检索具有特定元数据属性的文档。
- **语法**：
  ```python
  {
      "metadata_field": {
          "$operator": "value"
      }
  }
  ```
- **操作符**:
  - `$eq`: 等于
  - `$ne`: 不等于
  - `$gt`: 大于
  - `$gte`: 大于等于
  - `$lt`: 小于
  - `$lte`: 小于等于
  - `$in`: 属于列表中的值
  - `$nin`: 不属于列表中的值

### 逻辑运算符
- **$and**: 结合多个条件，文档需满足所有条件。
- **$or**: 结合多个条件，文档满足任一条件即可。

示例：
```python
{
    "$and": [
        {"category": {"$eq": "科技"}},
        {"published_date": {"$gte": "2023-01-01"}}
    ]
}
```
或
```python
{
    "$or": [
        {"title": {"$contains": "人工智能"}},
        {"body": {"$contains": "机器学习"}}
    ]
}
```
"""

In [678]:
def retrievalAgent(user_input):
    """Main function to interact with the SQL database based on user input."""
    tools = [
        {
        "type": "function",
        "function": {
            "name": "chroma_search",
            "description": "此函数基于Chroma向量存储，执行语义搜索并返回与用户问题相关的文档。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "description": "基于用户问题，执行语义搜索的查询内容。",
                        "type": "string"
                    },
                    "filter": {
                        "description": "基于chroma规则的过滤条件，用于筛选文档。",
                        "type": "json"
                    },
                },
                "required": ["query","filter"]
            },
        },
        }
    ]
    
    system_prompt = """
    你的职责是智能分析用户查询与元数据架构，调用chroma_search函数来搜索相关内容。
    首先，仔细解析用户的问题，识别出明确的查询条件与潜在的语义意图。
    随后，利用元数据结构信息：
    1. 确定哪些查询参数可以通过元数据字段进行精确筛选（例如，发布日期等）；
    2. 识别哪些信息需要运用语义搜索技术，从招标公告的非结构化文本中提炼。
    接下来，根据Chroma数据库的查询逻辑，精心构造查询指令。
    对于结构化，遵循Chroma的精确筛选规则，编写正确chroma_search函数的filter参数；
    而对于语义查询，利用Chroma的向量检索能力来找出最相关的文档段落，输入正确的query参数。
    
    
    
    """
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"用户的问题是: {user_input}"},
        {"role": "user", "content": f"元数据表的结构： {metadata_schema}"},
        {"role": "user", "content": f"chroma的查询筛选规则： {metadata_schema}"},
    ]
    
    import dashscope
    dashscope.api_key = api_key_qwen
    model_name = "qwen-plus"
    temperature = 0.2
    
    logger.info('AI model initialized: %s', model_name)
    
    raw_response = dashscope.Generation.call(
        messages=messages,
        model=model_name,
        tools=tools,
        temperature=temperature,
        result_format='message'
    )
    # 如果没有call 函数怎么处理？

    return raw_response


In [679]:
query="4月下旬，佛山市市政工程的标有哪些？他们开标时间都是什么时候"

response = retrievalAgent(query)

In [680]:
print(response.output.choices[0].message.content)
print(f'-'*20)

用户的问题主要包含两个查询条件：一是项目位于佛山市，二是招标公告在4月下旬发布。我们可以从元数据结构中看到，"province"字段可以用来筛选省份，但佛山是城市级别，因此我们需要使用"province"字段筛选广东省，然后在非结构化的招标公告原文中寻找关于"佛山市市政工程"的信息。"publish_date"字段可以用来筛选4月下旬的公告。

首先，我们构造精确筛选的条件，即发布日期在4月21日至4月30日之间（假设4月下旬指的是4月的最后10天），并且项目位于广东省。然后，我们需要对"佛山市市政工程"这个条件进行语义搜索，因为这是对公告内容的非结构化查询。

让我们先构造精确筛选的filter部分：

```json
{
  "filter": {
    "province": "广东省",
    "publish_date": {
      "$gte": "2022-04-21",
      "$lte": "2022-04-30"
    }
  }
}
```

接下来，构造语义查询的query部分，我们将用户的问题作为查询内容：

```json
{
  "query": "佛山市市政工程"
}
```

现在，我们可以调用`chroma_search`函数来执行查询。
--------------------


In [715]:
from langchain.docstore.document import Document, filter_complex_metadata
from langchain.vectorstores import Chroma
from langchain.embeddings import DashScopeEmbeddings
from datetime import datetime, date

def custom_filter_complex_metadata(documents, *, allowed_types=(str, int, float, bool, date)):
    """
    Filter out metadata types that are not supported for a vector store.
    """
    filtered_docs = filter_complex_metadata(documents, allowed_types=allowed_types)
    
    # Additional custom filtering logic here
    # For example, you can filter out documents with certain metadata values
    filtered_docs = [doc for doc in filtered_docs if doc.metadata.get("is_active", False)]
    
    return filtered_docs

# Create some documents with date metadata
documents = [
    Document(page_content="Document 1", metadata={"publish_date": date(2024, 2, 29), "rating": 4.5, "is_active": True}),
    Document(page_content="Document 2", metadata={"publish_date": date(2023, 5, 15), "rating": 3, "is_active": False}),
    Document(page_content="Document 3", metadata={"publish_date": date(2023, 6, 1), "category": "science fiction", "is_active": True}),
]

# Convert date metadata to string
for doc in documents:
    doc.metadata["publish_date"] = doc.metadata["publish_date"].isoformat()

# Filter the documents using the custom function
filtered_docs = custom_filter_complex_metadata(documents)

# Create a DashScopeEmbeddings instance
api_key = "your_dashscope_api_key"
embeddings = DashScopeEmbeddings(model="text-embedding-v1", dashscope_api_key=api_key)

# Create a Chroma vector store instance
chroma_store = Chroma(collection_name="my_collection", embedding_function=embeddings)

# Add the filtered documents to the Chroma vector store
chroma_store.add_documents(filtered_docs)

# Filter the documents by date range
start_date = datetime.strptime("2023-06-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2024-03-01", "%Y-%m-%d").date()

filtered_results = chroma_store.similarity_search(
    query="Document",
    filter={
        "publish_date": {
            "$gte": start_date,
            "$lte": end_date
        },
        "is_active": True
    }
)

# Print the filtered documents
for doc in filtered_results:
    print(f"Document: {doc.page_content}, Publish Date: {doc.metadata['publish_date']}, Is Active: {doc.metadata['is_active']}")


ValueError: Expected operand value to be an int or a float for operator $gte, got 2023-06-01